### Database Parsing and Processing

SQL Database

In [1]:
import os
import sqlite3
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader
from typing import List
from langchain_core.documents import Document

In [2]:
# Create a directory
database_dir = os.makedirs('data/db', exist_ok=True)
database_dir

In [3]:
# Create sample sqlite database
conn = sqlite3.connect('data/db/company_db.db')
cursor = conn.cursor()

In [4]:
# Create employees tables 
cursor.execute('''
               CREATE TABLE IF NOT EXISTS employees
               (id INTEGER PRIMAY KEY, 
               name TEXT,
               role TEXT,
               department TEXT,
               age INTEGER,
               gender TEXT,
               salary REAL)
               ''')

In [5]:
# Create projects tables 
cursor.execute('''
               CREATE TABLE IF NOT EXISTS projects
               (id INTEGER PRIMAY KEY, 
               name TEXT,
               status TEXT,
               department TEXT,
               budget REAL,
               priority TEXT,
               manager_id INTEGER)
               ''')

In [6]:
# Create sample data for the employees table
employees = [
    (1, 'John Smith', 'Data Analyst', 'IT', 30, 'Male', 75000.00),
    (2, 'Jane Doe', 'Project Manager', 'Operations', 45, 'Female', 95000.00),
    (3, 'Peter Jones', 'Software Engineer', 'IT', 28, 'Male', 85000.00),
    (4, 'Sarah Lee', 'HR Specialist', 'Human Resources', 34, 'Female', 65000.00),
    (5, 'Mike Brown', 'Financial Analyst', 'Finance', 40, 'Male', 110000.00),
    (6, 'Emily Davis', 'Marketing Coordinator', 'Marketing', 25, 'Female', 58000.00),
    (7, 'Chris Evans', 'Database Administrator', 'IT', 50, 'Male', 120000.00),
    (8, 'Jessica White', 'Recruiter', 'Human Resources', 32, 'Female', 72000.00),
    (9, 'David Wilson', 'Sales Manager', 'Sales', 48, 'Male', 130000.00),
    (10, 'Olivia Martin', 'Graphic Designer', 'Marketing', 29, 'Female', 62000.00)
]

projects = [
    (1, 'Website Redesign', 'In Progress', 'IT', 55000.00, 'High', 3),
    (2, 'Mobile App Development', 'Completed', 'IT', 150000.00, 'High', 2),
    (3, 'Internal Database Migration', 'Planning', 'IT', 80000.00, 'Medium', 7),
    (4, 'Cloud Infrastructure Upgrade', 'In Progress', 'IT', 250000.00, 'High', 1),
    (5, 'Marketing Campaign Launch', 'Completed', 'Marketing', 30000.00, 'Medium', 6),
    (6, 'New Product Research', 'Planning', 'R&D', 10000.00, 'Low', 5),
    (7, 'Cybersecurity Audit', 'In Progress', 'IT', 75000.00, 'High', 7),
    (8, 'Employee Training Portal', 'Completed', 'Human Resources', 45000.00, 'Medium', 4),
    (9, 'Sales Funnel Optimization', 'In Progress', 'Sales', 60000.00, 'High', 9),
    (10, 'Corporate Rebranding', 'Planning', 'Marketing', 90000.00, 'Low', 10)
]

In [7]:
# Insert the data into the table
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?, ?, ?, ?, ?, ?, ?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?, ?, ?, ?, ?, ?, ?)', projects)


In [8]:
cursor.execute('SELECT * FROM employees')

In [9]:
conn.commit()
conn.close()

Databse Content Extraction

In [10]:
# Method 1: Using SQLDATABASES utility

db = SQLDatabase.from_uri('sqlite:///data/db/company_db.db')

# Extract database information
print(f'Tables: {db.get_usable_table_names()}')
print(f'\nTable DDL:')
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	age INTEGER, 
	gender TEXT, 
	salary REAL
)

/*
3 rows from employees table:
id	name	role	department	age	gender	salary
1	John Smith	Data Analyst	IT	30	Male	75000.0
2	Jane Doe	Project Manager	Operations	45	Female	95000.0
3	Peter Jones	Software Engineer	IT	28	Male	85000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	department TEXT, 
	budget REAL, 
	priority TEXT, 
	manager_id INTEGER
)

/*
3 rows from projects table:
id	name	status	department	budget	priority	manager_id
1	Website Redesign	In Progress	IT	55000.0	High	3
2	Mobile App Development	Completed	IT	150000.0	High	2
3	Internal Database Migration	Planning	IT	80000.0	Medium	7
*/


In [11]:
# Method 2: Conversion of Custom SQL to Document 
print('Custom SQL Processing')

def sql_to_documents(db_path:str)-> List[Document]:
    '''Conversion of SQL Database to documents with context'''
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []
    
    # Create documents for each table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    for table in tables:
        table_name = table[0]
        
        # Get the table schema
        cursor.execute(f'PRAGMA table_info({table_name});')
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]
        
        # Get the table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        
        #Create table overviews document
        table_content = f'Table: {table_name}\n'
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f'Total Records: {len(rows)}\n\n'
        
        # Add sample records
        table_content += 'Sample Records:\n'
        for row in rows[:5]: # Get the first 5 rows
            record = dict(zip(column_names, row))
            table_content += f'{record}\n'
            
        doc = Document(
            page_content=table_content,
            metadata = {
                'source': db_path,
                'table_name': table_name,
                'row_records': len(rows),
                'data_type': 'sql_table'
            }
        )
        
        documents.append(doc)
    return documents

Custom SQL Processing


In [12]:
sql_to_documents('data/db/company_db.db')

[Document(metadata={'source': 'data/db/company_db.db', 'table_name': 'employees', 'row_records': 10, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, age, gender, salary\nTotal Records: 10\n\nSample Records:\n{'id': 1, 'name': 'John Smith', 'role': 'Data Analyst', 'department': 'IT', 'age': 30, 'gender': 'Male', 'salary': 75000.0}\n{'id': 2, 'name': 'Jane Doe', 'role': 'Project Manager', 'department': 'Operations', 'age': 45, 'gender': 'Female', 'salary': 95000.0}\n{'id': 3, 'name': 'Peter Jones', 'role': 'Software Engineer', 'department': 'IT', 'age': 28, 'gender': 'Male', 'salary': 85000.0}\n{'id': 4, 'name': 'Sarah Lee', 'role': 'HR Specialist', 'department': 'Human Resources', 'age': 34, 'gender': 'Female', 'salary': 65000.0}\n{'id': 5, 'name': 'Mike Brown', 'role': 'Financial Analyst', 'department': 'Finance', 'age': 40, 'gender': 'Male', 'salary': 110000.0}\n"),
 Document(metadata={'source': 'data/db/company_db.db', 'table_name': 'pr

In [13]:
for sql_doc in sql_to_documents('data/db/company_db.db'):
    print(sql_doc)

page_content='Table: employees
Columns: id, name, role, department, age, gender, salary
Total Records: 10

Sample Records:
{'id': 1, 'name': 'John Smith', 'role': 'Data Analyst', 'department': 'IT', 'age': 30, 'gender': 'Male', 'salary': 75000.0}
{'id': 2, 'name': 'Jane Doe', 'role': 'Project Manager', 'department': 'Operations', 'age': 45, 'gender': 'Female', 'salary': 95000.0}
{'id': 3, 'name': 'Peter Jones', 'role': 'Software Engineer', 'department': 'IT', 'age': 28, 'gender': 'Male', 'salary': 85000.0}
{'id': 4, 'name': 'Sarah Lee', 'role': 'HR Specialist', 'department': 'Human Resources', 'age': 34, 'gender': 'Female', 'salary': 65000.0}
{'id': 5, 'name': 'Mike Brown', 'role': 'Financial Analyst', 'department': 'Finance', 'age': 40, 'gender': 'Male', 'salary': 110000.0}
' metadata={'source': 'data/db/company_db.db', 'table_name': 'employees', 'row_records': 10, 'data_type': 'sql_table'}
page_content='Table: projects
Columns: id, name, status, department, budget, priority, manager_